# Analyzing Hacker News

## Understanding the performance of Ask HN vs Show HN posts

Hacker news is a site started by the startup incubator Y Combinator, where user-submitted stories (known as 'posts') are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We will be using a (reduced) public data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts). The data set we are analyzing is only 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are the colums:

1. id - The unique identifier from Hacker News for the post
2. title - The title of the post
3. url - The URL that the posts links to, if it the post has a URL
4. num_points - The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
6. num_comments - The number of comments that were made on the post
7. author - The username of the person who submitted the post
8. created_at - The date and time at which the post was submitted

In [1]:
# import the dataset
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5]) 

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [2]:
# remove the headers
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Since our specific interest is only in "Ask HN" and "Show HN" posts, we will be creating a new list of list containing just the data for those titles:

## Extracting Ask HN and Show HN Posts

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


## Calculating the Average Number of Comments for Ask HN and Show HN posts

In [4]:
# computing the average number of comments on ask posts
total_ask_comments = 0

for post in ask_posts:
    total_ask_comments += int(post[4])
      
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)
    

14.038417431192661


In [5]:
# computing the average number of comments on show posts
total_show_comments = 0

for post in show_posts:
    total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


This calculation proves that, on average, "Ask HN" posts receive more engagement in regards to comments than "Show HN". 

#### For all posts with comments in the Hacker News database:

1. "Ask HN" posts reveived an average of 14 comments
2. "Show HN" posts received an average of 10 comments

## Finding the Amount of Ask Posts and Comments by Hour Created

Since ask posts are more likely to receive comments, we'll focus the remainder of our analysis on just these posts.

Next, we will determine the following:
1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [6]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append(
        [row[6], int(row[4])]
    )
    
counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")

    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment
    else: 
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment
        


We created:
1. counts_by_hour: contains the number of ask posts created during each hour of the day
2. comments_by_hour: contains the corresponding number of comments ask posts created at each hour received

## Calculating the Average Number of Comments for Ask HN Posts by hour

In [7]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['19', 10.8],
 ['21', 16.009174311926607],
 ['16', 16.796296296296298],
 ['18', 13.20183486238532],
 ['13', 14.741176470588234],
 ['12', 9.41095890410959],
 ['09', 5.5777777777777775],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['15', 38.5948275862069],
 ['23', 7.985294117647059],
 ['10', 13.440677966101696],
 ['01', 11.383333333333333],
 ['11', 11.051724137931034],
 ['17', 11.46],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['02', 23.810344827586206],
 ['14', 13.233644859813085],
 ['05', 10.08695652173913],
 ['03', 7.796296296296297],
 ['04', 7.170212765957447],
 ['07', 7.852941176470588],
 ['20', 21.525]]

## Sorting and Printing Values from a List of Lists

In [8]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

# Sort the values and print the the 5 hours with the highest average comments.
print("Top 5 Hours for 'Ask HN' Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )
    


[[10.8, '19'], [16.009174311926607, '21'], [16.796296296296298, '16'], [13.20183486238532, '18'], [14.741176470588234, '13'], [9.41095890410959, '12'], [5.5777777777777775, '09'], [6.746478873239437, '22'], [10.25, '08'], [38.5948275862069, '15'], [7.985294117647059, '23'], [13.440677966101696, '10'], [11.383333333333333, '01'], [11.051724137931034, '11'], [11.46, '17'], [8.127272727272727, '00'], [9.022727272727273, '06'], [23.810344827586206, '02'], [13.233644859813085, '14'], [10.08695652173913, '05'], [7.796296296296297, '03'], [7.170212765957447, '04'], [7.852941176470588, '07'], [21.525, '20']]
Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.